In [4]:
import requests
import re
import pandas as pd

In [2]:
html = requests.get('https://www.alta.ru/currency/').text

In [3]:
def get_voltage():
    html = requests.get('https://www.alta.ru/currency/').text

    # нашли таблицы с валютами(основную и побочную), запарсили их
    # к себе, соединили и положили в переменную
    tables = pd.concat([pd.read_html(html)[7], pd.read_html(html)[6]],
                       ignore_index=True)

    # для простоты переименовываем столбцы на английские названия
    tables.columns = ['key', 'currency', 'rate']

    # разделяем ключи-цифры и ключи-имя валют (первый столбец)
    table_key = tables['key'].str.split(' ', expand=True)
    table_key.columns = ['number_key', 'name_key']

    # совмещаем разделённые столбцы с всей таблицей и удаляем столбец "повторюшку"
    tables = (pd.concat([table_key, tables], axis=1)).drop('key',
                                                           axis=1)

    # делаем табличку с наименованием валюты(где также лежит "цена за X ед." и её стоимостью (табл. 1)
    table_course = pd.concat([tables['currency'], tables['rate']],
                             axis=1)
    # взяли столбец с наименованием и разбили его на два(табл. 2):
    # столбец с именем и столбец "цена за Х ед."
    table_currency = table_course['currency']
    table_currency = table_currency.str.split('  ', expand=True)
    table_currency.columns = ['name_currency', 'multiplier']
    for n in range(149):
        table_currency['multiplier'][n] = int(''.join(re.findall('\d+',
                                                                 table_currency['multiplier'][n])))

    # объединили табл1 и табл2 удалив "повторюшку"
    table_course = (pd.concat([table_currency, table_course], axis=1)).drop('currency',
                                                                            axis=1)
    # в "цена валюты" сделали сумму за 1 ед. валюты
    table_course['rate'] = table_course['rate']/table_course['multiplier']

    # убрали столбец "цена за Х ед." и сделали общую таблицу df
    table_course = table_course.drop('multiplier', axis=1)
    df = pd.concat([table_key, table_course], axis=1)
    return df

In [5]:
def get_period(fromDate, toDate, Rcode):
    url = f'https://cbr.ru/currency_base/dynamics/?UniDbQuery.Posted=True&UniDbQuery.mode=1&UniDbQuery.date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ={Rcode}&UniDbQuery.From={fromDate}&UniDbQuery.To={toDate}'
    html = requests.get(url).text
#     print(url)
    table = ((pd.concat(pd.read_html(html), ignore_index = True)).drop(index=[0,1])).reset_index(drop=True)
    table.columns = ['date', 'multiplier', 'rate']
    table['rate'], table['multiplier'] = pd.to_numeric(table['rate'],downcast = "float"), pd.to_numeric(table['multiplier'])
    table['date'] = pd.to_datetime(table['date'],format="%d.%m.%Y")
    # for n in range(table.shape[0]):
    table['rate'] /= table['multiplier']*10000
    table = table.drop('multiplier', axis=1)
    table = table.iloc[::-1].reset_index(drop=True)
    return table

In [6]:
def getRcode(message):
    df = pd.read_csv('file1.csv', delimiter=',')
    df.drop('Unnamed: 0', axis=1)
    dfName = df.set_index('Name')
    dfEngName = df.set_index('EngName')
    try:
        return str(dfName.loc[message, 'Rcode'])[:6]
    except KeyError:
        try:
            return str(dfEngName.loc[message, 'Rcode'])[:6]
        except KeyError:
            return "Нет такой валюты"

In [7]:
df = pd.read_csv('data_182.csv',sep=';')

In [8]:
df_usd = df[df['letter_code']=='USD']

In [9]:
df_eur = df[df['letter_code']=='EUR']

In [10]:
df_usd = df_usd[df_usd['date']>='2000-01-01']

In [11]:
df_eur = df_eur[df_eur['date']>='2000-01-01']

In [12]:
df_eur_val = pd.DataFrame(df_eur['rate'])

In [13]:
past_days = 67
for day in range(past_days):
    d = day + 1
    df_eur_val[f"eur_back_{d}d"] = df_eur_val["rate"].shift(d)
df_eur_val.dropna(inplace=True)   
df_eur_val = df_eur_val.rename(columns={'rate': 'eur_rate'})
df_eur_val = df_eur_val.reset_index()
del df_eur_val['index']

In [14]:
df_usd_val = pd.DataFrame(df_usd['rate'])

In [15]:
past_days = 67
for day in range(past_days):
    d = day + 1
    df_usd_val[f"usd_back_{d}d"] = df_usd_val["rate"].shift(d)
df_usd_val.dropna(inplace=True)   
df_usd_val = df_usd_val.rename(columns={'rate': 'usd_rate'})
df_usd_val = df_usd_val.reset_index()
del df_usd_val['index']

In [16]:
eur_y = df_eur_val.iloc[:,0:7]
usd_y = df_usd_val.iloc[:,0:7]

In [17]:
eur_x = df_eur_val.iloc[:,7:]
usd_x = df_usd_val.iloc[:,7:]

In [18]:
Y = eur_y.join(usd_y)

In [19]:
X = eur_x.join(usd_x)

In [20]:
from sklearn.model_selection import train_test_split
X_1, X_2, y_1, y_2 = train_test_split(X, Y, test_size=0.3, random_state=42)

In [35]:
import tensorflow as tf
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1024, activation='relu',input_shape=(122,)))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(14, activation='linear'))

# model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-2)
model.compile(loss='mse', 
                   optimizer=optimizer, metrics=["accuracy"])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 1024)              125952    
                                                                 
 dense_14 (Dense)            (None, 512)               524800    
                                                                 
 dense_15 (Dense)            (None, 512)               262656    
                                                                 
 dense_16 (Dense)            (None, 14)                7182      
                                                                 
Total params: 920,590
Trainable params: 920,590
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x=X_1, y=y_1, 
          batch_size=128, epochs=15,
         validation_data=(X_2,y_2))

Epoch 1/15
44/44 [==============================] - 1s 9ms/step - loss: 26558938.0000 - accuracy: 0.1620 - val_loss: 1007.6695 - val_accuracy: 0.2550
Epoch 2/15
44/44 [==============================] - 0s 7ms/step - loss: 399.2976 - accuracy: 0.1747 - val_loss: 329.2314 - val_accuracy: 0.0995
Epoch 3/15
44/44 [==============================] - 0s 7ms/step - loss: 331.8997 - accuracy: 0.1555 - val_loss: 330.2914 - val_accuracy: 0.2550
Epoch 4/15
44/44 [==============================] - 0s 7ms/step - loss: 330.7021 - accuracy: 0.1669 - val_loss: 329.6548 - val_accuracy: 0.2550
Epoch 5/15
44/44 [==============================] - 0s 7ms/step - loss: 331.2447 - accuracy: 0.1590 - val_loss: 330.8645 - val_accuracy: 0.0796
Epoch 6/15
44/44 [==============================] - 0s 7ms/step - loss: 331.0465 - accuracy: 0.1588 - val_loss: 330.9145 - val_accuracy: 0.1049
Epoch 7/15
44/44 [==============================] - 0s 7ms/step - loss: 331.6380 - accuracy: 0.1814 - val_loss: 329.3965 - val_acc

In [32]:
model.evaluate(X_2,y_2)

75/75 [==============================] - 0s 3ms/step - loss: 478.4872 - accuracy: 0.1521


[478.4871520996094, 0.15212811529636383]

In [24]:
from matplotlib import pyplot as plt

In [25]:
from keras.models import load_model

In [26]:
nn = load_model('Predict_neural_network_v-1.h5')